# Calculo de tiempos promedios de transacciones por sucursal
Para este calculo se tomara de referencia las transacciones realizadas por hora y se calculara el tiempo de espera entre cada transaccion.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('facturas.csv')

### Promedio de tiempo inverido en las facturas
Puntos a considerar:
- Solo tomar la Caja_Id igual a 1
- Dividir por numero de sucursal de esta forma nos data un tiempo promedio por sucursal por cada factura
- Se tomara como muestra la hora pico es decir la hora donde hay mas facturas
- Columna Factura_FechaHora
- Columna Factura_Fecha

In [5]:
# cantidad de filas
print(len(df))
# Tomar solo la Caja 1
df = df[df['Caja_Id'] == 1]

# Tomar solo Emp_Id = 1
df = df[df['Emp_Id'] == 1]

# cantidad de filas
print(len(df))

1488891
305661


In [74]:

df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')
df_grouped = df.groupby(
    [df['Suc_Id'], df['Caja_Id'], df['Factura_Fecha'], df['Factura_FechaHora'].dt.hour]
)['Factura_Id'].count().reset_index(name='Cantidad_Facturas')

In [53]:

df_grouped.describe()
print(df_grouped)

       Suc_Id  Caja_Id Factura_Fecha  Factura_FechaHora  Cantidad_Facturas
0           1        1     1/10/2024                 13                  1
1           1        1     1/10/2024                 14                  1
2           1        1     1/10/2024                 15                  2
3           1        1     1/10/2024                 19                  1
4           1        1    10/10/2024                 11                  2
...       ...      ...           ...                ...                ...
71055     328        1     9/10/2024                 10                  1
71056     328        1     9/10/2024                 15                  1
71057     328        1     9/10/2024                 16                  1
71058     328        1     9/10/2024                 17                  1
71059     328        1     9/10/2024                 18                  1

[71060 rows x 5 columns]


### Tiempos promedio de transacciones por sucursal, eliminar los que esten por debajo de la media
Puntos a considerar:
- Se tomara como muestra la hora pico es decir la hora donde hay mas facturas


In [75]:
# Calcular los valores mas altos los que esten por encima del 80% de cantidad de facturas por hora en cada sucursal, caja y fecha
df_grouped['value'] = df_grouped.groupby(['Suc_Id', 'Caja_Id', 'Factura_Fecha'])['Cantidad_Facturas'].transform(
    lambda x: x.quantile(0.95))
print(df_grouped)

df_grouped = df_grouped[df_grouped['Cantidad_Facturas'] > df_grouped['value']]
print(df_grouped)




       Suc_Id  Caja_Id Factura_Fecha  Factura_FechaHora  Cantidad_Facturas  \
0           1        1     1/10/2024                 13                  1   
1           1        1     1/10/2024                 14                  1   
2           1        1     1/10/2024                 15                  2   
3           1        1     1/10/2024                 19                  1   
4           1        1    10/10/2024                 11                  2   
...       ...      ...           ...                ...                ...   
71055     328        1     9/10/2024                 10                  1   
71056     328        1     9/10/2024                 15                  1   
71057     328        1     9/10/2024                 16                  1   
71058     328        1     9/10/2024                 17                  1   
71059     328        1     9/10/2024                 18                  1   

       value  
0       1.85  
1       1.85  
2       1.85  
3  

In [76]:
df_grouped.describe()

,Suc_Id,Caja_Id,Factura_FechaHora,Cantidad_Facturas,value
count,5742.000000,5742.0,5742.000000,5742.000000,5742.000000
mean,153.974922,1.0,14.053988,9.079415,8.032393
std,93.285036,0.0,3.911921,4.444274,3.894986
min,1.000000,1.0,0.000000,2.000000,1.600000
25%,75.000000,1.0,11.000000,6.000000,5.500000
50%,147.000000,1.0,14.000000,8.000000,7.400000
75%,240.000000,1.0,17.000000,11.000000,9.800000
max,328.000000,1.0,23.000000,47.000000,43.800000


In [77]:
# Agrupar por sucursal y hora, y calcular la mediana de la cantidad de facturas por hora
mediana_por_sucursal = df_grouped.groupby(['Suc_Id'])['Cantidad_Facturas'].median().reset_index()

# Renombrar la columna para mayor claridad
mediana_por_sucursal.rename(columns={'Cantidad_Facturas': 'Mediana_Facturas_Hora'}, inplace=True)

print(mediana_por_sucursal)


     Suc_Id  Mediana_Facturas_Hora
0         1                    2.5
1         2                    5.0
2         3                   11.0
3         4                    8.5
4         5                   10.0
..      ...                    ...
282     324                    4.0
283     325                    4.0
284     326                    7.0
285     327                    8.0
286     328                    5.0

[287 rows x 2 columns]


In [78]:
mediana_por_sucursal.describe()

,Suc_Id,Mediana_Facturas_Hora
count,287.000000,287.000000
mean,155.470383,8.385017
std,93.941127,3.305381
min,1.000000,2.000000
25%,75.500000,6.000000
50%,150.000000,8.000000
75%,240.500000,10.000000
max,328.000000,28.000000


In [1]:
import pandas as pd

# Datos
data = [
    ["KIELSA CLINICA TP SIN PLAN", "003-003-01-00071998", "6:43:00 AM", "6:58:00 AM", "12:15:00 AM"],
    ["KIELSA CLINICA TE SIN PLAN", "003-005-01-00045772", "12:00:00 AM", "12:00:00 AM", "12:00:00 AM"],
    ["KIELSA CLINICA TE SIN PLAN", "003-003-01-00072118", "9:35:00 AM", "9:40:00 AM", "12:05:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-003-06-00008091", "2:10:00 AM", "2:15:00 AM", "12:05:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-003-01-00072172", "2:10:00 AM", "2:17:00 AM", "12:07:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-005-01-00045988", "10:49:00 AM", "10:56:00 AM", "12:07:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-005-01-00046127", "5:39:00 AM", "5:42:00 AM", "12:03:00 AM"],
    ["KIELSA CLINICA TE SIN PLAN", "003-003-01-00072646", "7:36:00 AM", "7:54:00 AM", "12:18:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-003-01-00072685", "5:51:00 AM", "6:17:00 AM", "12:26:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-005-01-00046219", "11:50:00 AM", "12:03:00 PM", "12:13:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-003-01-00072761", "5:28:00 AM", "5:52:00 AM", "12:24:00 AM"],
    ["KIELSA CLINICA TE SIN PLAN", "003-005-01-00046250", "5:43:00 AM", "6:03:00 AM", "12:20:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-003-01-00072796", "7:30:00 AM", "7:37:00 AM", "12:07:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-005-01-00046478", "4:58:00 AM", "5:07:00 AM", "12:09:00 AM"],
    ["KIELSA CLINICA TP SIN PLAN", "003-005-01-00046515", "9:28:00 AM", "9:34:00 AM", "12:06:00 AM"],
]

# Crear DataFrame
df = pd.DataFrame(data, columns=["Type", "Code", "StartTime", "EndTime", "OtherTime"])

# Convertir horas a formato datetime y calcular duraciones
df["StartTime"] = pd.to_datetime(df["StartTime"], format="%I:%M:%S %p")
df["EndTime"] = pd.to_datetime(df["EndTime"], format="%I:%M:%S %p")
df["Duration"] = (df["EndTime"] - df["StartTime"]).dt.total_seconds() / 60  # Duración en minutos

# Separar TP y TE
tp = df[df["Type"].str.contains("TP")]
te = df[df["Type"].str.contains("TE")]

# Calcular tiempo promedio
tp_avg = tp["Duration"].mean()
te_avg = te["Duration"].mean()

# Resultados
print(f"Tiempo promedio TP: {tp_avg:.2f} minutos")
print(f"Tiempo promedio TE: {te_avg:.2f} minutos")

Tiempo promedio TP: 11.09 minutos
Tiempo promedio TE: 10.75 minutos
